In [ ]:
import numpy as np
from keras.callbacks import LambdaCallback
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.models import Sequential
from keras.optimizers import RMSprop
import nltk
from nltk.corpus import gutenberg
import math

txt_files=nltk.corpus.gutenberg.fileids()[0:5]
char_corpus = gutenberg.raw(txt_files)


chars = sorted(list(set(char_corpus)))
print('total number of chars:', len(chars))
c2i = dict((c, i) for i, c in enumerate(chars))
i2c = dict((i, c) for i, c in enumerate(chars))

length = 50+1
sentences = []
next_chars = []
for i in range(0, len(char_corpus) - length):
    sentences.append(char_corpus[i: i + length])
    next_chars.append(char_corpus[i + length])

x = np.zeros((len(sentences), length, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, c2i[char]] = 1
    y[i, c2i[next_chars[i]]] = 1

    

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.1)


   
    
    
# build the model: a single LSTM
print('Training char model on 100 epochs on 5 files')
model = Sequential()
model.add(LSTM(128, input_shape=(length, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def on_epoch_end(epoch, logs):
    model_json = model.to_json()
    with open("char_model{}.json".format(epoch), "w") as json_file:
        json_file.write(model_json)
    model.save_weights("char_model{}.h5".format(epoch))


model.fit(x, y, batch_size=128, epochs=100,callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

loss=model.evaluate(xtest,ytest,batch_size=128, verbose=1)
print("perperxility is:",math.exp(loss))
    
    

total number of chars: 82
